In [ ]:
import os
import re
from typing import List
from dotenv import load_dotenv
from typing_extensions import TypedDict

# LangChain 및 Upstage 관련 모듈
from langchain_upstage import ChatUpstage
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field

# LangGraph 관련 모듈
from langgraph.graph import StateGraph, END

# --- 1. 초기 설정 (LLM, Embeddings, Vector DB 로드) ---
print(">>> 1단계: 초기 설정을 시작합니다...")
load_dotenv()

# LLM을 ChatUpstage로 설정
llm = ChatUpstage(model_name="solar-1-mini-chat", temperature=0)

# 💥 임베딩 모델도 Upstage로 통일!
embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

# 이전에 만들어둔 FAISS 벡터 DB 로드
db_path = "./db/cafe_db"
if not os.path.exists(db_path):
    raise FileNotFoundError(f"'{db_path}' 경로에 벡터 DB가 없습니다. 이전 문제의 DB 구축 스크립트를 먼저 실행해주세요.")

# Upstage 임베딩으로 생성된 DB를 로드해야 합니다.
# 만약 DB가 OpenAI 임베딩으로 만들어졌다면 DB 구축 스크립트부터 다시 실행해야 합니다.
vectorstore = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
print("초기 설정 완료!")


# --- 2. Graph 상태 정의 (수정된 부분) ---
# add_messages 와 Annotated 를 사용하지 않고 간단한 list로 정의합니다.
class GraphState(TypedDict):
    messages: list

# --- 3. Graph 노드(Node) 함수 정의 (수정된 부분) ---

class RouteQuery(BaseModel):
    route: str = Field(description="주어진 질문에 가장 적합한 경로. 'menu', 'price', 'recommend', 'general' 중 하나여야 합니다.")

def classify_input(state: GraphState):
    print("\n[노드: classify_input]")
    question = state["messages"][-1].content
    
    structured_llm = llm.with_structured_output(RouteQuery)
    prompt = ChatPromptTemplate.from_messages([
        ("system", "사용자 질문의 의도를 파악하여 'menu', 'price', 'recommend', 'general' 네 가지 중 하나로 분류해주세요."),
        ("human", "{question}")
    ])
    
    router_chain = prompt | structured_llm
    result = router_chain.invoke({"question": question})
    
    print(f"'{question}' -> 분류: {result.route}")
    
    # 💥 수정: 대화 목록을 수동으로 관리합니다.
    # 기존 메시지 목록에 새 메시지를 추가하여 반환합니다.
    new_message = AIMessage(content="", additional_kwargs={"route": result.route})
    return {"messages": state["messages"] + [new_message]}

# 각 노드의 return 부분을 수정하여 메시지 리스트를 직접 업데이트합니다.
def search_menu(state: GraphState):
    print("\n[노드: search_menu]")
    question = state["messages"][-2].content
    retrieved_docs = retriever.invoke(question)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    prompt = ChatPromptTemplate.from_template("아래 정보를 바탕으로 질문에 답해주세요:\n\n{context}\n\n질문: {question}")
    rag_chain = prompt | llm | StrOutputParser()
    response = rag_chain.invoke({"context": context, "question": question})
    
    # 💥 수정: 대화 목록을 수동으로 관리합니다.
    new_message = AIMessage(content=response)
    return {"messages": state["messages"] + [new_message]}

def search_price(state: GraphState):
    print("\n[노드: search_price]")
    question = state["messages"][-2].content
    retrieved_docs = vectorstore.similarity_search("메뉴 가격", k=5)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    prompt = ChatPromptTemplate.from_template("아래 가격 정보를 바탕으로 질문에 답해주세요:\n\n{context}\n\n질문: {question}")
    rag_chain = prompt | llm | StrOutputParser()
    response = rag_chain.invoke({"context": context, "question": question})

    # 💥 수정: 대화 목록을 수동으로 관리합니다.
    new_message = AIMessage(content=response)
    return {"messages": state["messages"] + [new_message]}

def recommend_menu(state: GraphState):
    print("\n[노드: recommend_menu]")
    question = state["messages"][-2].content
    retrieved_docs = retriever.invoke(question)
    if not retrieved_docs:
        retrieved_docs = vectorstore.similarity_search("인기 메뉴", k=3)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    prompt = ChatPromptTemplate.from_template("아래 메뉴 정보를 바탕으로 메뉴를 추천해주세요:\n\n{context}\n\n질문: {question}")
    rag_chain = prompt | llm | StrOutputParser()
    response = rag_chain.invoke({"context": context, "question": question})

    # 💥 수정: 대화 목록을 수동으로 관리합니다.
    new_message = AIMessage(content=response)
    return {"messages": state["messages"] + [new_message]}

def general_response(state: GraphState):
    print("\n[노드: general_response]")
    question = state["messages"][-2].content
    prompt = ChatPromptTemplate.from_template("당신은 카페 어시스턴트입니다. 일반적인 대화를 나눠주세요.\n\n질문: {question}")
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke({"question": question})

    # 💥 수정: 대화 목록을 수동으로 관리합니다.
    new_message = AIMessage(content=response)
    return {"messages": state["messages"] + [new_message]}

# --- 4. Graph 생성 및 엣지(Edge) 연결 ---

def decide_next_node(state: GraphState):
    route = state["messages"][-1].additional_kwargs.get("route", "general")
    if route == "menu": return "search_menu"
    elif route == "price": return "search_price"
    elif route == "recommend": return "recommend_menu"
    else: return "general_response"

workflow = StateGraph(GraphState)
workflow.add_node("classifier", classify_input)
workflow.add_node("search_menu", search_menu)
workflow.add_node("search_price", search_price)
workflow.add_node("recommend_menu", recommend_menu)
workflow.add_node("general_response", general_response)
workflow.set_entry_point("classifier")
workflow.add_conditional_edges("classifier", decide_next_node, {
    "search_menu": "search_menu", "search_price": "search_price",
    "recommend_menu": "recommend_menu", "general_response": "general_response",
})
workflow.add_edge("search_menu", END)
workflow.add_edge("search_price", END)
workflow.add_edge("recommend_menu", END)
workflow.add_edge("general_response", END)

# --- 5. Graph 컴파일 및 실행 ---
app = workflow.compile()

# 대화 시작
while True:
    user_input = input("USER: ")
    if user_input.lower() in ["quit", "exit"]:
        break
    
    response = app.invoke({"messages": [HumanMessage(content=user_input)]})
    print("ASSISTANT:", response["messages"][-1].content)

>>> 1단계: 초기 설정을 시작합니다...
초기 설정 완료!

[노드: classify_input]
'인기메뉴' -> 분류: menu

[노드: search_menu]
ASSISTANT: 인기 메뉴로는 다음과 같은 커피들이 있습니다:

1. **아메리카노**: ₩4,500
   - 원두 본연의 맛을 가장 잘 느낄 수 있는 클래식한 블랙 커피입니다.

2. **카페라떼**: ₩5,500
   - 대표적인 밀크 커피로 크리미한 질감과 부드러운 맛이 특징입니다.

3. **카푸치노**: ₩5,000
   - 이탈리아 전통 커피로 진한 커피 맛과 부드러운 우유 거품의 조화가 일품입니다.

4. **바닐라 라떼**: ₩6,000
   - 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴로, 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러집니다.

5. **카라멜 마키아토**: ₩6,500
   - 스팀 밀크 위에 에스프레소를 부어 만든 후 카라멜 시럽과 휘핑크림으로 마무리한 달콤한 커피입니다.

6. **프라푸치노**: ₩7,000
   - 에스프레소와 우유, 얼음을 블렌더에 갈아 만든 시원한 음료로, 부드럽고 크리미한 질감이 특징입니다.

7. **티라미수**: ₩7,500
   - 이탈리아 전통 디저트로 부드럽고 달콤한 맛이 특징입니다.

이 메뉴들은 각각의 독특한 맛과 특징을 가지고 있어 많은 사람들에게 사랑받고 있습니다.

[노드: classify_input]
'아메리카 가격' -> 분류: price

[노드: search_price]
ASSISTANT: 아메리카노의 가격은 ₩4,500입니다.

[노드: classify_input]
'끝' -> 분류: general

[노드: general_response]
ASSISTANT: 안녕하세요! 카페 어시스턴트로서 여러분이 편안하고 즐거운 시간을 보내실 수 있도록 도와드리겠습니다. 어떤 음료를 주문하시겠어요? 커피, 차, 스무디 등 다양한 메뉴가 준비되어 있습니다. 또한, 간식이나 디저트도 함께 추천해 드릴 수 있